In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
from collections import Counter
import ast  

Move integron.filter.tsv file to another dir

In [ ]:
path='Path/to/bacant_result/'
output='Path/to/integron_result/'
all_folder=os.listdir(path)
for i in all_folder:
    if os.path.exists(path+i+"/integron.filter.tsv"):
        shutil.copyfile(path+i+"/integron.filter.tsv", output+i+'.tsv')

Move transposon.filter.tsv file to another dir

In [ ]:
path='/node05/program/zhouhaiyang/ints/back/bacant/800_ifaE_bacant_result/'
output='/node05/program/zhouhaiyang/ints/back/bacant/transposon/'
all_folder=os.listdir(path)
for i in all_folder:
    if os.path.exists(path+i+"/transposon.filter.tsv"):
        if os.path.getsize(path+i+"/transposon.filter.tsv") > 162:
            shutil.copyfile(path+i+"/transposon.filter.tsv", output+i+'.tsv')

Motifi transposon/integron file

In [ ]:
########   Transposon/integron Motifi  #######
transposon_path='/node05/program/zhouhaiyang/ints/bacant/integron/'
transposon_out='/node05/program/zhouhaiyang/ints/bacant/Motifi_integron/'
transposon_list=os.listdir(transposon_path)

for i in transposon_list:
    new_column=[]
    file=pd.read_csv(transposon_path+i,sep='\t')
#####Change the name by your need********
    file = pd.DataFrame({
        "Contig": file["QUERY"],
        "Transposon_Type": file["INTEGRON TYPE|ACCESSION"],
        "Start": file["QUERY START"],
        "End": file["QUERY END"]})
    
    file["Location"] = file["Start"].astype(str).str.cat(file["End"].astype(str), sep="-")
    file=file.drop("Start", axis=1)
    file=file.drop("End",axis=1)
#####For Transposon Motifi, code bellow is needed********
    file_contig=[]
    temp_file_contig=file['Contig'].to_list()
    for ii in temp_file_contig:
        ii=ii.split('~')[0]
        file_contig.append(ii)
    file['Contig']=file_contig

#####Change the name by your need********
    file_transposon=[]
    temp_file_transposon=file['Transposon_Type'].to_list()
    for ii in temp_file_transposon:
        ii=ii.split('|')[0]
        file_transposon.append(ii)
    
    file["Transposon_Type"]=file_transposon
    file.to_csv(transposon_out+i,index=False)

Locate ARG on Transposon/integron

In [ ]:
def Search_Location(search_contig,filename,RE_location):
    global integron_RE
    global integron_index
    integron_RE=0
    integron_index='No_Transposon'
    # print('RE_Location is '+ RE_location)
    integron_path='/node05/program/zhouhaiyang/ints/bacant/Motifi_integron/'
    file=pd.read_csv(integron_path+filename)
    file=file[file['Contig']==search_contig]    
    file_integron_location=file['Location'].to_list()   #######

    for temp_location in file_integron_location:
        prophage_start=temp_location.split('-')[0]
        prophage_start=int(prophage_start)
        if prophage_start-5000 < 0:
            prophage_start=0
        else:
            prophage_start=prophage_start-5000
        prophage_start=int(prophage_start)
        
        prophage_end=temp_location.split('-')[-1]
        prophage_end=int(prophage_end)
        prophage_end=prophage_end+5000
        prophage_end=int(prophage_end)
        
        RE_start=RE_location.split('-')[0]  #######
        RE_start=int(RE_start)
        RE_end=RE_location.split('-')[-1]  ##########
        RE_end=int(RE_end)
        if prophage_start <= RE_start and RE_end <= prophage_end:
            integron_RE=1
            integron_index=file[file['Location'] == temp_location].index[0]


In [ ]:
ARG_path='/Path/to/abricate/result/'
ARG_path_list=os.listdir(ARG_path)

integron_path='Path/to/Motifi_transposon/'
path_list=os.listdir(integron_path)


print('Start Process Files: '+ str(len(path_list)))


Strain=[]
ARG_Location_on_integron=[]
ARG_type=[]
integron_type=[]

for i in path_list:
    print(i)
    Integron_file=i
    i=i.split('.')[0]
    i=[x for x in ARG_path_list if x.startswith(i)][0]

    ARG_motify=pd.read_csv(ARG_path+i,sep='\t')
    RE_motify_contig=ARG_motify['SEQUENCE'].to_list()  ###contig
    RE_motify_start=ARG_motify['START'].to_list()  #####333-444
    RE_motify_end=ARG_motify['END'].to_list()
    RE_motify_RE_name=ARG_motify['GENE'].to_list() ######RE_name

    counter=list(range(len(RE_motify_contig)))
    final_count=[]
    temp_integron_type=[]
    
    for ii in counter:
        search_contig=RE_motify_contig[ii]
        RE_location=str(RE_motify_start[ii])+'-'+str(RE_motify_end[ii])
        Search_Location(search_contig,Integron_file,RE_location)
        final_count.append(integron_RE)
        temp_integron_type.append(integron_index)
        
    if all(item == "No_Transposon" for item in temp_integron_type):
        integron_type.append('No_Transposon carrier')
    
    else:
        integron_type_temp=[]  #############
        numbers_only=[item for item in temp_integron_type if item != "No_Transposon"]
        integron_step2_file=pd.read_csv(integron_path+Integron_file)
        integron_step2_file=integron_step2_file.iloc[:, -2].to_list()
        for iiii in numbers_only:
            temp_iiii=integron_step2_file[iiii]
            integron_type_temp.append(temp_iiii)
        integron_type.append(integron_type_temp)

    if 1 in final_count:
        temp_RE_type=[]
        final_count_index_1=[index for index, value in enumerate(final_count) if value == 1]
        for iii in final_count_index_1:
            temp_RE_type.append(RE_motify_RE_name[iii])
        ARG_type.append(temp_RE_type)
    else:
        ARG_type.append("No_Transposon-borne ARG")

    Strain.append(Integron_file.replace('.tsv',''))


# print(len(RE_Location_on_integron))
print(len(Strain))
print(len(ARG_type))
print(len(integron_type))
  
  
# Define the output directory and filename  
output_dir = 'Path/to/co-location-result/'  
output_filename = 'result.csv'  
output_path = output_dir + output_filename  
  
# Create a dictionary with the data  
data = {  
    'Strain': Strain,  
    'ARG_type': ARG_type,  
    'integron_type': integron_type  
}  
  
df = pd.DataFrame(data)  
df.to_csv(output_path, index=False)  

